In [1]:
!pip install gradio
!pip install grad-cam
!pip install pytorch-lightning
!pip install torch-lr-finder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [2]:
import os
import sys

import numpy as np
import gradio as gr
from PIL import Image

import torch, torchvision
from torchvision import transforms

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

if 'google.colab' in sys.modules:
    # Download the repo from GitHub
    print("Downloading repository on Colab...")
    !git clone https://github.com/Shilpaj1994/ERA.git

    # Import files from the downloaded repository
    sys.path.insert(0,'./ERA/Session12/')

from resnet import LITResNet
from visualize import FeatureMapVisualizer

Mounted at /content/drive/
Cloning into 'ERA'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 401 (delta 5), reused 9 (delta 3), pack-reused 389
Receiving objects: 100% (401/401), 33.53 MiB | 38.03 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [4]:
dir = '/content/drive/MyDrive/ERA/Assignments/Session-12/'
model_path = '/content/drive/MyDrive/ERA/Assignments/Session-12/model_23.ckpt'

In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

model = LITResNet.load_from_checkpoint(model_path, map_location=torch.device('cpu'), strict=False, class_names=classes)
model.eval()

LITResNet(
  (inv_normalize): Normalize(mean=[-2.1739130434782608, -2.1739130434782608, -2.1739130434782608], std=[4.3478260869565215, 4.3478260869565215, 4.3478260869565215])
  (accuracy): MulticlassAccuracy()
  (prep_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (custom_block1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (resnet_block1): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, k

In [6]:
model

LITResNet(
  (inv_normalize): Normalize(mean=[-2.1739130434782608, -2.1739130434782608, -2.1739130434782608], std=[4.3478260869565215, 4.3478260869565215, 4.3478260869565215])
  (accuracy): MulticlassAccuracy()
  (prep_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (custom_block1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (resnet_block1): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, k

## GradCam

In [7]:
from collections import OrderedDict

def inference(input_img,
              transparency=0.5,
              number_of_top_classes=3,
              target_layer_number=4):
    """
    Function to run inference on the input image
    :param input_img: Image provided by the user
    :parma transparency:
    :param number_of_top_classes:
    :param target_layer_number:
    """
    # Calculate mean over each channel of input image
    mean_r, mean_g, mean_b = np.mean(input_img[:, :, 0]/255.), np.mean(input_img[:, :, 1]/255.), np.mean(input_img[:, :, 2]/255.)

    # Calculate Standard deviation over each channel
    std_r, std_g, std_b = np.std(input_img[:, :, 0]/255.), np.std(input_img[:, :, 1]/255.), np.std(input_img[:, :, 2]/255.)

    # Convert img to tensor and normalize it
    _transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean_r, mean_g, mean_b), (std_r, std_g, std_b))
        ])

    # Save a copy of input img
    org_img = input_img

    # Apply the transforms on the input image
    input_img = _transform(input_img)

    # Add batch dimension to perform inference
    input_img = input_img.unsqueeze(0)

    # Get Model Predictions
    with torch.no_grad():
        outputs = model(input_img)
        o = torch.exp(outputs)[0]
        confidences = {classes[i]: float(o[i]) for i in range(10)}

    # Select the top classes based on user input
    sorted_confidences = sorted(confidences.items(), key=lambda val: val[1], reverse=True)
    show_confidences = OrderedDict(sorted_confidences[:number_of_top_classes])

    # Name of layers defined in the model
    _layers = ['prep_layer', 'custom_block1', 'resnet_block1',
               'custom_block2', 'custom_block3', 'resnet_block3']
    target_layers = [eval(f'model.{_layers[target_layer_number-1]}[0]')]

    # Get the class activations from the selected layer
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
    grayscale_cam = cam(input_tensor=input_img, targets=None)
    grayscale_cam = grayscale_cam[0, :]

    # Overlay input image with Class activations
    visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
    return show_confidences, visualization

## Misclassified Images

In [8]:
def display_misclassified_images(number: int = 1):
    """
    Display the misclassified images saved during training
    :param number: Number of images to display
    """
    # List to store names of misclassified images
    data = []

    # Get the names of all the files from Misclassified directory
    file_names = os.listdir(dir+'misclassified/')

    # Save the correct name and misclassified class name as a tuple in the `data` list
    for file in file_names:
        file_name, extension = file.split('.')
        correct_label, misclassified = file_name.split('_')
        data.append((correct_label, misclassified))

    # Create a path to the images for Gradio to access them
    file_path = [dir+'misclassified/'+file for file in file_names]

    # Return the file path and names of correct and misclassified images
    return file_path[:number], data[:number]

## Feature Maps and Kernels

In [9]:
# Create an object of the Class
viz = FeatureMapVisualizer(model)

def feature_maps(input_img, kernel_number=32):
    """
    Function to return feature maps for the selected image
    :param kernel_number: Number of kernel in all 6 layers
    """
    # Calculate mean over each channel of input image
    mean_r, mean_g, mean_b = np.mean(input_img[:, :, 0]/255.), np.mean(input_img[:, :, 1]/255.), np.mean(input_img[:, :, 2]/255.)

    # Calculate Standard deviation over each channel
    std_r, std_g, std_b = np.std(input_img[:, :, 0]/255.), np.std(input_img[:, :, 1]/255.), np.std(input_img[:, :, 2]/255.)

    # Convert img to tensor and normalize it
    _transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean_r, mean_g, mean_b), (std_r, std_g, std_b))
        ])

    # Apply transforms on the input image
    input_img = _transform(input_img)

    # Visualize feature maps for kernel number 32
    plt = viz.visualize_feature_map_of_kernel(image=input_img, kernel_number=kernel_number)
    return plt

def get_kernels(layer_number):
    """
    Function to get the kernels from the layer
    :param layer_number: Number of layer from which kernels to be visualized
    """
    # Visualize kernels from layer
    plt = viz.visualize_kernels_from_layer(layer_number=layer_number)
    return plt

## App

In [16]:
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # CIFAR10 trained on ResNet18 Model
           - A model architecture by [David C](https://github.com/davidcpage) which is trained on CIFAR10 for 24 Epochs to achieve accuracy of 90+%
           - One Cycle Policy is used during training to speed up the trainig process
           - The model works for following classes: `plane`, `car`, `bird`, `cat`, `deer`, `dog`, `frog`, `horse`, `ship`, `truck`

        ### A simple Gradio interface
        - To infer what exactly the model is looking at using GradCAM results
        - To display the misclassified images from the 10% of test data of CIFAR10 dataset
        - To visualize the feature maps from each of the six convolutional block's first layer
        - To visualize the kernels from each of the six convolutional block's first layer
        """
    )

    # #############################################################################
    # ################################ GradCam Tab ################################
    # #############################################################################
    with gr.Tab("GradCam"):
        with gr.Row():
            img_input = [gr.Image(shape=(32, 32), label="Input Image")]
            gradcam_outputs = [gr.Label(),
                               gr.Image(shape=(32, 32), label="Output").style(width=128, height=128)]

        with gr.Row():
            gradcam_inputs = [gr.Slider(0, 1, value=0.5, label="How much percentage overlap of what model is looking at in the image?"),
                              gr.Slider(1, 10, value=3, step=1, label="How many top predictions you want to see?"),
                              gr.Slider(1, 6, value=4, step=1, label="From 6 layers of the model, which layer's class activation you want to see?")]

        gradcam_button = gr.Button("Submit")
        gradcam_button.click(inference, inputs=img_input + gradcam_inputs, outputs=gradcam_outputs)

        gr.Markdown("## Examples")
        gr.Examples([dir+'dog.jpg', dir+'cat.jpg', dir+'frog.jpg', dir+'bird.jpg', dir+'shark-plane.jpg',
                    dir+'car.jpg', dir+'truck.jpg', dir+'horse.jpg', dir+'plane.jpg', dir+'ship.png'], inputs=img_input, fn=inference)

    # ###########################################################################################
    # ################################ Misclassified Images Tab #################################
    # ###########################################################################################
    with gr.Tab("Misclassified Images"):
        with gr.Row():
            mis_inputs = [gr.Slider(1, 10, value=1, step=1, label="Select the Number of Misclassified Images you want to see")]
            mis_outputs = [gr.Gallery(label="Misclassified Images", show_label=False, elem_id="gallery").style(columns=[2], rows=[2], object_fit="contain", height="auto"),
                           gr.Dataframe(headers=["Correct Label", "Misclassified Label"], type="array", datatype="str", row_count=10, col_count=2)]
        mis_button = gr.Button("Display Misclassified Images")
        mis_button.click(display_misclassified_images, inputs=mis_inputs, outputs=mis_outputs)


    # ################################################################################################
    # ################################ Feature Maps Visualization Tab ################################
    # ################################################################################################
    with gr.Tab("Feature Map Visualization"):
        with gr.Column():
            feature_map_input = [gr.Image(shape=(32, 32), label="Feature Map Input Image"), gr.Slider(1, 32, value = 16, step=1, label="Select a Kernel number for which features maps from all 6 layers to be shown")]
            map = gr.Plot().style()
            feature_map_button = gr.Button("Visualize FeatureMaps")
        feature_map_button.click(feature_maps, inputs=feature_map_input, outputs=map)

    # ##########################################################################################
    # ################################ Kernel Visualization Tab ################################
    # ##########################################################################################
    with gr.Tab("Kernel Visualization"):
        with gr.Column():
            kernel_input = [gr.Slider(1, 4, value=2, step=1, label="Select a layer number from which the kernels to be shown")]
            map = gr.Plot().style()
            kernel_button = gr.Button("Visualize Kernels")

        kernel_button.click(get_kernels, inputs=kernel_input, outputs=map)

gr.close_all()
demo.launch(debug=True, share=True)

<ipython-input-16-92a1c2be998b>:24: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  gr.Image(shape=(32, 32), label="Output").style(width=128, height=128)]
<ipython-input-16-92a1c2be998b>:44: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  mis_outputs = [gr.Gallery(label="Misclassified Images", show_label=False, elem_id="gallery").style(columns=[2], rows=[2], object_fit="contain", height="auto"),
<ipython-input-16-92a1c2be998b>:56: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  map = gr.Plot().style()
<ipython-input-16-92a1c2be998b>:66: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  map = gr.Plot().style()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a288fa27ac7b319375.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a288fa27ac7b319375.gradio.live
